In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image
import os
import pandas as pd

In [ ]:
# Loading pre trained resnet50
res50 = models.resnet50(pretrained=True)

In [10]:
# Loading self trained resnet50
#gpu
#res50=torch.load("dl_models/ensemble_model/level2/res50(balanced)(pw)32-120(better).pth")
#cpu
res50=torch.load("dl_models/ensemble_model/level2/res50(balanced)(pw)32-120(better).pth",map_location=torch.device('cpu'))
#or map_location="cpu"

In [11]:
res50.fc.out_features

2

In [ ]:
#creating list of imagenet classes for classification purposes
#Use this part only when imagenet is used as a dataset i.e. basically a pre trained pytorch model. 
f = open("imagenet_classes.txt", "r")
all_cls=list(f)
cls=[]
for cl in all_cls:
    m=cl.split(',')[1]
    cls.append(m)

In [12]:
# Iterating the database folder to get the names of all the images contained in folder
imgset=[]
for (root, dirs, files) in os.walk('database/',topdown=True):
    for f in files:
        imgset.append(f)
imgset.sort()
print(imgset)
print('\nTotal {} images'.format(len(imgset)))

['0.jpg', '1.jpg', '10.jpg', '11.jpg', '12.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']

Total 13 images


In [13]:
#Defining the transforms that are to be applied to the image for inference.

transform=transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225])
])

In [14]:
tensors=[]      #List of tensors of images present in database folder
for im in imgset:
    img=Image.open(os.path.join('database',im))   #Opening image using PIL 
    img_t=transform(img)
    batch_t=torch.unsqueeze(img_t,0)
    tensors.append(batch_t)

In [15]:
tensors
#main=torch.stack(tensors)   #creates tensor of tensors

[tensor([[[[1.1872, 1.2214, 1.2385,  ..., 0.7933, 0.7248, 0.7248],
           [1.1015, 1.1529, 1.2214,  ..., 0.7933, 0.7077, 0.7077],
           [1.0331, 1.1358, 1.2214,  ..., 0.8104, 0.7248, 0.7248],
           ...,
           [0.0569, 0.0741, 0.0741,  ..., 0.0227, 0.0569, 0.0569],
           [0.0569, 0.0741, 0.0741,  ..., 0.0227, 0.0569, 0.0569],
           [0.0569, 0.0741, 0.0741,  ..., 0.0227, 0.0398, 0.0398]],
 
          [[1.5707, 1.6057, 1.6232,  ..., 1.2556, 1.1856, 1.1856],
           [1.4832, 1.5357, 1.6057,  ..., 1.2556, 1.1681, 1.1681],
           [1.4132, 1.5182, 1.6057,  ..., 1.2731, 1.1856, 1.1856],
           ...,
           [0.2752, 0.2927, 0.2927,  ..., 0.2227, 0.2577, 0.2577],
           [0.2752, 0.2927, 0.2927,  ..., 0.2402, 0.2752, 0.2752],
           [0.2752, 0.2927, 0.2927,  ..., 0.2402, 0.2577, 0.2577]],
 
          [[1.9254, 1.9603, 1.9777,  ..., 1.6814, 1.6117, 1.6117],
           [1.8383, 1.8905, 1.9603,  ..., 1.6814, 1.5942, 1.5942],
           [1.7685, 1.87

In [17]:
#if torch.cuda.is_available():    
#clslist=[]
probslist=[]
res50.eval()
for t in tensors:
    #b = t.to(torch.device("cuda"))   #send the tensor to gpu i.e. cuda, basically converting tensor to cuda tensor.
    b = t.to(torch.device("cpu"))   #for cpu
    out=res50(b)
    percentage=torch.nn.functional.softmax(out, dim=1)[0] * 100  #calcuating percentage from prob*100 using softmax.
    print(percentage)
    for value in percentage:
        probslist.append(float(value))
    #use in case of imagenet dataset, i.e. when using pytorch pretrained model only(remove above two lines in that case, and uncomment the clslist)
    #_, indices = torch.sort(out, descending=True)
    #[(cls[idx], probs[idx].item()) for idx in indices[0][:5]]
    #for idx in indices[0][:4]:
    #    clslist.append(cls[idx])
    #    probslist.append(float(probs[idx]))
    #    print(cls[idx], probs[idx].item())

tensor([99.6859,  0.3141], grad_fn=<MulBackward0>)
tensor([1.0000e+02, 1.0376e-04], grad_fn=<MulBackward0>)
tensor([9.9962e+01, 3.8246e-02], grad_fn=<MulBackward0>)
tensor([98.3926,  1.6074], grad_fn=<MulBackward0>)
tensor([99.8985,  0.1015], grad_fn=<MulBackward0>)
tensor([99.7143,  0.2857], grad_fn=<MulBackward0>)
tensor([99.7840,  0.2160], grad_fn=<MulBackward0>)
tensor([89.5472, 10.4528], grad_fn=<MulBackward0>)
tensor([65.2501, 34.7499], grad_fn=<MulBackward0>)
tensor([27.7438, 72.2562], grad_fn=<MulBackward0>)
tensor([34.3715, 65.6285], grad_fn=<MulBackward0>)
tensor([9.9959e+01, 4.0505e-02], grad_fn=<MulBackward0>)
tensor([53.4991, 46.5009], grad_fn=<MulBackward0>)


In [18]:
probslist     # list of probabilities of all the images with all 4 classes in one list

[99.68594360351562,
 0.31405287981033325,
 99.99989318847656,
 0.00010375904093962163,
 99.96174621582031,
 0.03824599087238312,
 98.39262390136719,
 1.6073753833770752,
 99.89852142333984,
 0.1014758050441742,
 99.71426391601562,
 0.285734623670578,
 99.78396606445312,
 0.21602527797222137,
 89.54716491699219,
 10.452834129333496,
 65.25009155273438,
 34.749908447265625,
 27.743772506713867,
 72.25623321533203,
 34.3714599609375,
 65.6285400390625,
 99.9594955444336,
 0.04050545021891594,
 53.49907302856445,
 46.50092697143555]

In [ ]:
blst=[]
plst=[]
vlst=[]
wlst=[]
#above 4 list contain probabilies of each class separately.
i=0
while i<len(probslist):
    #blst.append(probslist[i])
    plst.append(probslist[i])
    #vlst.append(probslist[i+2])
    wlst.append(probslist[i+1])
    i=i+2

In [ ]:
#df=pd.DataFrame({'Imagename':imgset,'building':blst,'person':plst,'vehicle':vlst,'weapon':wlst})
df=pd.DataFrame({'Imagename':imgset,'person':plst,'weapon':wlst})
df

In [ ]:
df.to_csv("log_data res50(b)(pw)32-120(better).csv", index=None, header=True)